In [0]:
spark.conf.set("fs.azure.account.auth.type.atharvk9999storage.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.atharvk9999storage.dfs.core.windows.net", 
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.atharvk9999storage.dfs.core.windows.net", 
               "Client ID")  # client ID
spark.conf.set("fs.azure.account.oauth2.client.secret.atharvk9999storage.dfs.core.windows.net", 
               "Client Secret") # secret
spark.conf.set("fs.azure.account.oauth2.client.endpoint.atharvk9999storage.dfs.core.windows.net", 
               "https://login.microsoftonline.com/Tenant_Id/oauth2/token")  # tenant ID

In [0]:
adls_path = "abfss://rawfiles@atharvk9999storage.dfs.core.windows.net/"

In [0]:
jdbc_url = "jdbc:sqlserver://azuresqlserver9999.database.windows.net:1433;database=azuresqldb"
db_props = {
    "user": "username",
    "password": "password",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
table_name="Sales"

csv_temp_path = adls_path + f"csvfiles_temp/{table_name}/"
csv_final_path = adls_path + f"csvfiles/{table_name}.csv"

df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=db_props)

df.coalesce(1).write.mode("overwrite") \
    .option("header", True) \
    .csv(csv_temp_path)

files = dbutils.fs.ls(csv_temp_path)
csv_file = [f for f in files if f.name.endswith(".csv")][0]

dbutils.fs.mv(csv_file.path, csv_final_path)
dbutils.fs.rm(csv_temp_path, recurse=True)

print(f"{table_name} table exported as single CSV to: {csv_final_path}")

Sales table exported as single CSV to: abfss://rawfiles@atharvk9999storage.dfs.core.windows.net/csvfiles/Sales.csv


In [0]:
df_csv = spark.read.option("header", True).csv(csv_input_path)

In [0]:
#Write Parquet as single file
df_csv.coalesce(1).write.mode("overwrite").parquet(parquet_temp_path)

files = dbutils.fs.ls(parquet_temp_path)
parquet_file = [f for f in files if f.name.endswith(".parquet")][0]

dbutils.fs.mv(parquet_file.path, parquet_final_path)
dbutils.fs.rm(parquet_temp_path, recurse=True)

True

In [0]:
#Write Avro as single file
df_csv.coalesce(1).write.mode("overwrite").format("avro").save(avro_temp_path)

files = dbutils.fs.ls(avro_temp_path)
avro_file = [f for f in files if f.name.endswith(".avro")][0]

dbutils.fs.mv(avro_file.path, avro_final_path)
dbutils.fs.rm(avro_temp_path, recurse=True)

True

In [0]:
print("Parquet and Avro stored as single files:")
print(f"Parquet: {parquet_final_path}")
print(f"Avro   : {avro_final_path}")

Parquet and Avro stored as single files:
Parquet: abfss://rawfiles@atharvk9999storage.dfs.core.windows.net/parquetfiles/Sales.parquet
Avro   : abfss://rawfiles@atharvk9999storage.dfs.core.windows.net/avrofiles/Sales.avro


In [0]:
parquet_temp_path = adls_path + "parquetfiles_temp"
avro_temp_path = adls_path + "avrofiles_temp"
csv_temp_path = adls_path + "csvfiles_temp"

for temp_path in [csv_temp_path, parquet_temp_path, avro_temp_path]:
    try:
        dbutils.fs.rm(temp_path, recurse=True)
        print(f"Deleted: {temp_path}")
    except Exception as e:
        print(f"Could not delete {temp_path}: {str(e)}")

Deleted: abfss://rawfiles@atharvk9999storage.dfs.core.windows.net/csvfiles_temp
Deleted: abfss://rawfiles@atharvk9999storage.dfs.core.windows.net/parquetfiles_temp
Deleted: abfss://rawfiles@atharvk9999storage.dfs.core.windows.net/avrofiles_temp
